In [2]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import wordnet
from random import randint

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/gaurav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/gaurav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def synonym_replacement(sentence, n):
    old_sentence = sentence
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in nltk.corpus.stopwords.words('english')]))
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = synonyms[0]
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n: # n is the number of words to be replaced
            break

    sentence = ' '.join(new_words)
    if old_sentence == sentence:
        flag = False
    else:
        flag = True
    return flag, sentence

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char.isalpha() or char == ' '])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [3]:
df = pd.read_csv('preReviewsCSV.csv')
df.dropna(subset=['reviewContent', 'label'], inplace=True)
df1 = df[df.label == 1]
# df2 = df[df.label == 0].sample(n=df1.shape[0], random_state=1)
# new_df = pd.concat([df1, df2], ignore_index=True)
# data = new_df.sample(frac=1, random_state=2)

# train_data = data.sample(frac=0.8, random_state=1)
# val_data = data.drop(train_data.index)

In [4]:
cnt = 0
fake_df = pd.DataFrame(columns=['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating',
       'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID',
       'label', 'doc_embeddings', 'Polarity'])
percentages = [0.1, 0.2, 0.3, 0.5, 0.7, 0.9]
for i, row in df1.iterrows():
    sent = row['reviewContent']
    new_sents = []
    for p in percentages:
        flag, new_sent = synonym_replacement(sent, int(p*len(sent.split(' '))))
        if flag:
            cnt += 1
            new_sents.append(new_sent)
    new_sents = list(set(new_sents)) + [sent]

    for s in new_sents:
        new_row = {'date': row['date'], 'reviewID': row['reviewID'], 'reviewerID': row['reviewerID'], 'reviewContent': s, 'rating': row['rating'], 
                   'usefulCount': row['usefulCount'], 'coolCount': row['coolCount'], 'funnyCount': row['funnyCount'], 'flagged': row['flagged'], 
                   'restaurantID': row['restaurantID'], 'label': row['label'], 'doc_embeddings': row['doc_embeddings'], 'Polarity': row['Polarity']}
        fake_df = pd.concat([fake_df, pd.DataFrame([new_row])], ignore_index=True)
            
    print(f"{len(fake_df)}/{i+1}", end = '\r')

In [10]:
fake_df.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,restaurantID,label,doc_embeddings,Polarity
0,8/25/2012,XgqV3Ac0CLg2R-_c-yZvTw,tfn2U8XfLaJnRD9EXpTxLQ,food for thought amaz two ticket tomorrow dark...,5,0,0,0,Y,pbEiXam9YJL3neCYHGwLUA,1,[-0.01494107 0.79770064 0.57676816 0.156622...,0.7184
1,8/25/2012,XgqV3Ac0CLg2R-_c-yZvTw,tfn2U8XfLaJnRD9EXpTxLQ,food for thought amaz slate tomorrow dark bil...,5,0,0,0,Y,pbEiXam9YJL3neCYHGwLUA,1,[-0.01494107 0.79770064 0.57676816 0.156622...,0.7184
2,8/25/2012,XgqV3Ac0CLg2R-_c-yZvTw,tfn2U8XfLaJnRD9EXpTxLQ,food amaz two ticket tomorrow night sunday any...,5,0,0,0,Y,pbEiXam9YJL3neCYHGwLUA,1,[-0.01494107 0.79770064 0.57676816 0.156622...,0.7184
3,8/25/2012,XgqV3Ac0CLg2R-_c-yZvTw,tfn2U8XfLaJnRD9EXpTxLQ,food amaz two ticket tomorrow dark sunday anyo...,5,0,0,0,Y,pbEiXam9YJL3neCYHGwLUA,1,[-0.01494107 0.79770064 0.57676816 0.156622...,0.7184
4,8/25/2012,XgqV3Ac0CLg2R-_c-yZvTw,tfn2U8XfLaJnRD9EXpTxLQ,food amaz two ticket tomorrow dark sunday anyo...,5,0,0,0,Y,pbEiXam9YJL3neCYHGwLUA,1,[-0.01494107 0.79770064 0.57676816 0.156622...,0.7184


In [11]:
fake_df.shape

(46747, 13)

In [12]:
df2 = df[df.label == 0].sample(n=fake_df.shape[0], random_state=123)

In [15]:
new_df = pd.concat([df2, fake_df], ignore_index=True).sample(frac=1, random_state=123)
new_df.to_csv('augmentedReviewsCSV.csv', index=False)

### Adding userCentric Features

In [3]:
df = pd.read_csv('augmentedReviewsCSV.csv')

In [7]:
df.columns

Index(['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating',
       'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID',
       'label', 'doc_embeddings', 'Polarity'],
      dtype='object')

##### Average review length per reviewer

In [9]:
df['review_length'] = df['reviewContent'].str.len()
avg_review_length_per_user = df.groupby('reviewerID')['review_length'].mean().reset_index(name='avg_review_length')
df = pd.merge(df, avg_review_length_per_user, on='reviewerID', how='left')

In [10]:
df.columns

Index(['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating',
       'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID',
       'label', 'doc_embeddings', 'Polarity', 'review_length',
       'avg_review_length'],
      dtype='object')

##### Std deviation of ratings of the reviewer's reviews

In [12]:
std_dev_ratings_per_user = df.groupby('reviewerID')['rating'].std().reset_index(name='std_dev_ratings')
df = pd.merge(df, std_dev_ratings_per_user, on='reviewerID', how='left')

In [13]:
df.columns

Index(['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating',
       'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID',
       'label', 'doc_embeddings', 'Polarity', 'review_length',
       'avg_review_length', 'std_dev_ratings'],
      dtype='object')

##### Max no. of reviews in a day

In [20]:
df['date'] = pd.to_datetime(df['date'], format = 'mixed', errors = 'coerce')
reviews_per_user_day = df.groupby(['reviewerID', df['date'].dt.date]).size().reset_index(name='daily_reviews')
max_reviews_per_user = reviews_per_user_day.groupby('reviewerID')['daily_reviews'].max().reset_index(name='max_daily_reviews')
df = pd.merge(df, max_reviews_per_user, on='reviewerID', how='left')

In [23]:
new_user_feat_included_df = df[['reviewContent', 'avg_review_length', 'std_dev_ratings', 'max_daily_reviews', 'label']]

In [24]:
new_user_feat_included_df.shape

(93494, 5)

In [25]:
new_user_feat_included_df.to_csv('newCSV.csv')